In [109]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [110]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle 
import warnings 
warnings.filterwarnings("ignore")


In [111]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [112]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents/intents.json').read()
intents = json.loads(data_file)

In [113]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [114]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [115]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [116]:
#lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))

#sort classes
classes = sorted(list(set(classes)))

#documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")

#classes = intents[tags]
print (len(classes), "classes\n", classes, "\n")

#words = all words, vocabulary
print (len(words), "unique lemmatized words", words, "\n")

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

In [117]:
#creating our training data 
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into list
random.shuffle(training)

# create train and test lists. X - patterns, Y - intents
train_x = [pattern[0] for pattern in training]
train_y = [pattern[1] for pattern in training]
print("Training data created")

Training data created


In [118]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [119]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer: ", model.layers[0].get_weights()[0])

First layer:  [[ 0.01303238 -0.0091239  -0.05345079 ...  0.04993492  0.07886007
   0.11661816]
 [-0.05252137 -0.10950106 -0.07925243 ... -0.06370482 -0.02023528
  -0.09624561]
 [ 0.07209764 -0.12359837 -0.11865751 ...  0.04251672  0.09357862
  -0.01773546]
 ...
 [-0.02214054  0.10086894 -0.00530582 ... -0.05970846  0.10861959
  -0.06564885]
 [-0.1221148   0.06878191 -0.00871807 ... -0.11519361  0.07842892
  -0.09312716]
 [-0.05848246 -0.07471838  0.03973584 ...  0.01700948 -0.01404786
   0.04391518]]


In [120]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [121]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0361 - loss: 3.6395   
Epoch 2/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step - accuracy: 0.1325 - loss: 3.5223
Epoch 3/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.1540 - loss: 3.3621
Epoch 4/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2078 - loss: 3.1539
Epoch 5/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3039 - loss: 2.7864
Epoch 6/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.3640 - loss: 2.5279
Epoch 7/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.3990 - loss: 2.3417 
Epoch 8/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step - accuracy: 0.4755 - loss: 2.0240
Epoch 9/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.5492 - loss: 1.8931
Epoch 10/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - accuracy: 0.6264 - loss: 1.6360
Epoch 11/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6490 - loss: 1.4198  
Epoch 12/100
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 

model created
